### Prepare dataset

Import necessary modules, and download tweets from database.

In [1]:
"""
EDA for database.
"""

import sys, os

sys.path.append(os.path.abspath(os.path.join("..", "..", "src")))

%load_ext autoreload
%autoreload 2
import re
import time

import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import widgets

import matplotlib
import pandas as pd
import numpy as np
from tqdm import tqdm

tqdm.pandas()

from common.database import Database
from common.app import App
from common.helpers import Helpers

app_run = App(debug=True)
db = Database("tweets.db", app=app_run)

In [2]:
with db:
    tws = db.get_all_tweets()
print(len(tws))
df_all = Helpers.df_from_db(tws)

238523


In [3]:
# Select tweets about covid
df_yes = Helpers.categorize_df_covid(df_all.copy())
print(f"{len(df_yes)=}")

# select those that are NOT coded
df_uncoded = df_yes[
    ~(df_yes["topic"].isin(Helpers.topics_cov))
    & ~(df_yes["topic"].isin(Helpers.topics_not_cov))
].copy()
print(f"{len(df_uncoded)=}")

# and those that are coded
df_coded = df_all[(df_all["topic"].isin(Helpers.topics_cov))].copy()
print(f"{len(df_coded)=}")

len(df_yes)=101640
len(df_uncoded)=16566
len(df_coded)=85074


In [4]:
# New way, should be used
df_yes = Helpers.categorize_df_covid(df_all.copy())
print(f"{len(df_yes)=}")

len(df_yes)=101640


In [5]:
# Convert date to be handled
df_uncoded["date"] = df_uncoded["created_at"].apply(Helpers.convert_date)
df_uncoded["date"] = pd.to_datetime(df_uncoded["date"], format="%d/%m/%Y")

df_coded["date"] = df_coded["created_at"].apply(Helpers.convert_date)
df_coded["date"] = pd.to_datetime(df_coded["date"], format="%d/%m/%Y")

df_all["date"] = df_all["created_at"].apply(Helpers.convert_date)
df_all["date"] = pd.to_datetime(df_all["date"], format="%d/%m/%Y")

df_yes["date"] = df_yes["created_at"].apply(Helpers.convert_date)
df_yes["date"] = pd.to_datetime(df_yes["date"], format="%d/%m/%Y")

In [6]:
# Only keep tweets in the time range
df_all_sorted = Helpers.sort_timerange(df_all.copy())
print(f"{len(df_all_sorted)=}") 

df_yes_sorted = Helpers.sort_timerange(df_yes.copy())
print(f"{len(df_yes_sorted)=}") 

df_uncoded_sorted = Helpers.sort_timerange(df_uncoded.copy())
print(f"{len(df_uncoded_sorted)=}") 

df_coded_sorted = Helpers.sort_timerange(df_coded.copy())
print(f"{len(df_coded_sorted)=}")

len(df_all_sorted)=185749
len(df_yes_sorted)=84233
len(df_uncoded_sorted)=2145
len(df_coded_sorted)=82088


### 1. General EDA about whole dataset

Period of interest: **01/01/2020 - 31/03/2021**

In [9]:
counts_all = df_all.groupby(["date"]).count()["tweet_id"]
counts_yes = df_yes.groupby(["date"]).count()["tweet_id"]
counts_coded = df_coded.groupby(["date"]).count()["tweet_id"]
counts_uncoded = df_uncoded.groupby(["date"]).count()["tweet_id"]

counts_all_sorted = df_all_sorted.groupby(["date"]).count()["tweet_id"]
counts_yes_sorted = df_yes_sorted.groupby(["date"]).count()["tweet_id"]
counts_coded_sorted = df_coded_sorted.groupby(["date"]).count()["tweet_id"]
counts_uncoded_sorted = df_uncoded_sorted.groupby(["date"]).count()["tweet_id"]

In [10]:
print("All retrieved tweets")
print(f"Total number of retrieved tweets: {len(df_all)}")
print(f"Total number of tweets about covid: {len(df_yes)}")
print(f"Total number of coded tweets: {len(df_coded)}")
print(f"Total number of uncoded tweets: {len(df_uncoded)}")

print("\nOver time range of interest")
print(f"Number of retrieved tweets in timerange: {len(df_all_sorted)}")
print(f"Number of tweets about covid in timerange: {len(df_yes_sorted)}")
print(f"Number of coded tweets in timerange: {len(df_coded_sorted)}")
print(f"Number of uncoded tweets in timerange: {len(df_uncoded_sorted)}")

All retrieved tweets
Total number of retrieved tweets: 238523
Total number of tweets about covid: 101640
Total number of coded tweets: 85074
Total number of uncoded tweets: 16566

Over time range of interest
Number of retrieved tweets in timerange: 185749
Number of tweets about covid in timerange: 84233
Number of coded tweets in timerange: 82088
Number of uncoded tweets in timerange: 2145


Note that new tweets from the actors are retrieved daily but not automatically added to the database. The real number (25.11.2021) is of ~350k tweets.

In [46]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=counts_all.index, y=counts_all, mode="lines", name="all retrieved tweets"))
fig.add_trace(go.Scatter(x=counts_yes.index, y=counts_yes, mode="lines", name="all tweets about covid"))
fig.add_trace(go.Scatter(x=counts_all.index, y=counts_all - counts_yes, mode="lines", name="difference"))  # recheck index

fig.add_vline(x="2020-01-01", line_color="red", line_dash="dash")
fig.add_vline(x="2021-03-31", line_color="red", line_dash="dash")
fig.update_layout(title="Tweets counts", xaxis_title="date", yaxis_title="count")
fig.show()

The cycle comes from the weeks. Actors tweet much less during the weekends.

#### 1.1 Spike of July 23

_In case this problem has already been solved:_  
_This section refers to a spike in number of tweets for 23th July 2020. See `descriptives/figures/july_23.png`._

In [25]:
# July 23
all_23 = df_all_sorted.loc[df_all_sorted["date"] == pd.to_datetime("2020-07-23")]
yes_23 = df_yes_sorted.loc[df_yes_sorted["date"] == pd.to_datetime("2020-07-23")]
display(all_23["handle"].value_counts().head(4))
display(yes_23["handle"].value_counts().head(4))

@WHO           664
@UN            252
@DrTedros      232
@WHO_Europe    170
Name: handle, dtype: int64

@WHO           660
@UN            250
@DrTedros      225
@WHO_Europe    166
Name: handle, dtype: int64

Out of the 1738 tweets that day, more than 1148 (66%) were tweeted by 3 accounts: @WHO, @UN and @DrTedros. We also see that almost all those tweets were about covid.

In [28]:
# Check type of tweets
display(yes_23.loc[yes_23["handle"] == "@WHO"]["type"].value_counts())

Reply      496
Retweet    100
New         64
Name: type, dtype: int64

In [35]:
yes_23.loc[(yes_23["handle"] == "@WHO") & (yes_23["type"] == "Reply") & (yes_23["old_text"].str.startswith("RY @WHO"))].sample(1)["url"].values

array(['https://twitter.com/WHO/status/1293498878617178112'], dtype=object)

In [33]:
# Tweets that are replies to WHO itself
ry_c = len(yes_23.loc[(yes_23["handle"] == "@WHO") & (yes_23["type"] == "Reply") & (yes_23["old_text"].str.startswith("RY @WHO"))])

print(f"Out of 496 replies, {ry_c} are replies to @WHO itself.")

Out of 496 replies, 495 are replies to @WHO itself.


As written above, out of 496 replies, 495 are replies to @WHO itself. However, one thing is bizarre. The `created_at` field corresponding to when a tweet is posted does not match the Twitter website. 

The tweet [1293498878617178112](https://twitter.com/WHO/status/1293498878617178112) was posted on August 12th 2020, also it is reported as July 23th in the database. It appears that the same thing happened with many tweets for this day. At this stage, it is unclear how and why this happened. As an example of a "normal" day, let us look at May 28th.

In [43]:
# May 28
all_28 = df_all_sorted.loc[df_all_sorted["date"] == pd.to_datetime("2020-05-28")]
yes_28 = df_yes_sorted.loc[df_yes_sorted["date"] == pd.to_datetime("2020-05-28")]

# Display counts
display(all_28["handle"].value_counts().head(4))
display(yes_28["handle"].value_counts().head(4))

# Sample a unique value and check date
print("URL:", all_28.sample(1)["url"].item())
print("Date:", all_28.sample(1)["date"].item())


@WHO_Europe    45
@DHSCgovuk     43
@EU_Health     39
@enmarchefr    32
Name: handle, dtype: int64

@DHSCgovuk         36
@WHO_Europe        32
@MinSoliSante      28
@gouvernementFR    25
Name: handle, dtype: int64

URL: https://twitter.com/BAG_OFSP_UFSP/status/1265979020585435137
Date: 2020-05-28 00:00:00


As expected, other days seem to correctly behave. The `created_at` field should be rechecked for every tweet with a non-missing `tweet_id`.

In [47]:
# TODO
# Check created_at field of tweets and update database
# see july_23.ipynb

#### 1.2 Uncoded tweets

In [30]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=counts_yes.index, y=counts_yes, mode="lines", name="all tweets about covid"))
fig.add_trace(go.Scatter(x=counts_coded.index, y=counts_coded, mode="lines", name="coded tweets"))
fig.add_trace(go.Scatter(x=counts_uncoded.index, y=counts_uncoded, mode="lines", name="uncoded tweets"))

fig.add_vline(x="2020-01-01", line_color="red", line_dash="dash")
fig.add_vline(x="2021-03-31", line_color="red", line_dash="dash")
fig.update_layout(title="Tweets counts", xaxis_title="date", yaxis_title="count")
fig.show()

We clearly see here that in the period of interest, almost all tweets that were retrieved and classified about covid were coded. The lacking ones are due to the automatic covid-classifier being updated. Some tweets that were previously not considered as being about covid now are and would need to be coded.  

In [32]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=counts_uncoded.index, y=counts_uncoded, mode="lines", name="uncoded tweets"))

fig.add_vline(x="2020-01-01", line_color="red", line_dash="dash")
fig.add_vline(x="2021-03-01", line_color="red", line_dash="dash")
fig.update_layout(title="Uncoded tweets count", xaxis_title="date", yaxis_title="count")
fig.show()

We can also look at the accounts that have the most uncoded tweets:

In [39]:
df_uncoded_sorted.groupby(["handle"]).count()["tweet_id"].sort_values(ascending=False).head(6)

handle
@BAG_OFSP_UFSP     494
@BR_Sprecher       177
@EDI_DFI           163
@DHSCgovuk         117
@gouvernementFR    110
@DrTedros           77
Name: tweet_id, dtype: int64

We clearly see that the accounts that have the most uncoded tweets are swiss. This is no surprise, as they also tweet in German and Italian, languages that were consequently (for the second wave) not considered. The rest are due to the classifier update.